In [79]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.applications import VGG16
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [62]:
# Chemins des dossiers
data_directory = r'C:\Users\skrfa\Desktop/Réseaux_de_neurones/malaria_hematie_dataset'
saines_directory = r'C:\Users\skrfa\Desktop/Réseaux_de_neurones/malaria_hematie_dataset/saines'
infectees_directory = r'C:\Users\skrfa\Desktop/Réseaux_de_neurones/malaria_hematie_dataset/infectees'

In [63]:
# Préprocesser les images
def preprocess_images(folder_path, label, target_size=(128, 128)):
    images = []
    labels = []
    for filename in tqdm(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), labels

In [64]:
# préprocesser les images infectees
infectees_images, infectees_labels = preprocess_images(r'C:\Users\skrfa\Desktop/Réseaux_de_neurones/malaria_hematie_dataset/infectees', 'infectees')

# préprocesserles images saines
saines_images, saines_labels = preprocess_images(r'C:\Users\skrfa\Desktop/Réseaux_de_neurones/malaria_hematie_dataset/saines', 'saines')

100%|███████████████████████████████████████████████████████████████████████████| 13779/13779 [00:14<00:00, 964.85it/s]


**Assurer que toutes les images ont la même shape permet de garantir que les données sont prêtes pour l'entraînement**

In [65]:
# shape des images
print("Image infectees Shapes:", set(img.shape for img in infectees_images))
print("Image Saines Shapes:", set(img.shape for img in saines_images))

Image infectees Shapes: {(128, 128, 3)}
Image Saines Shapes: {(128, 128, 3)}


In [66]:
# Concatenater les données Preprocesser
images = np.concatenate((infectees_images, saines_images))
labels = infectees_labels + saines_labels

In [67]:
# Normalisation des images
images_normalized = images / 255.0

In [68]:
# Encodage des labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

**Vérifier les shapes et le nombre des labels est necessaire pour verifier la cohérence des données et detecter les erreurs** 

In [69]:
# Vérifier les shapes et le nombre des labels  
print("Shape of Images:", images.shape)
print("Length of Labels:", len(labels_encoded))

Shape of Images: (27558, 128, 128, 3)
Length of Labels: 27558


**Data augmentation : est une technique utilisée pour augmenter la diversité de l'ensemble de données d'entraînement sans collecter de nouvelles données et pour reduire le surapprentissage**

In [71]:
# Augmentation des données
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

**Definir les dimensions des images et créer un modèle CNN en utilisant Keras pour la classification des images d'hématies infectées et saines**

In [74]:
# Détermination des dimensions des images prétraitées ==> Récupère les dimensions des images prétraitées (hauteur, largeur)
img_height, img_width, channels = images.shape[1:]

# Détermination du nombre de classes uniques ==> Détermine le nombre de classes uniques dans les labels encodés
num_classes = len(np.unique(labels_encoded))

# Construction du modèle CNN 
model = Sequential([
    # Première couche de convolution et pooling
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, channels)),
    MaxPooling2D((2, 2)),
    
    # Deuxième couche de convolution et pooling
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flattening
    Flatten(),
    
    # Couche dense avec dropout
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    # Couche de sortie
    Dense(num_classes, activation='softmax')
])

In [77]:
# Chargement du modèle VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Récuperation de l'encodeur pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Construction du modèle et ajouter des couches Dense
x = Flatten()(base_model.output)  # Flatten pour connecter à des couches Dense
x = Dense(128, activation='relu')(x)  # Couche Dense avec 128 neurones et activation ReLU
x = Dropout(0.5)(x)  # Dropout de 50% pour la régularisation
predictions = Dense(num_classes, activation='softmax')(x)  # Couche de sortie avec activation softmax

# Création du modèle final avec l'encodeur pré-entraîné et les nouvelles couches
model_vgg = Model(inputs=base_model.input, outputs=predictions)

In [80]:
# Définition des callbacks
callbacks = [
    EarlyStopping(patience=3, monitor='val_loss'),
    ReduceLROnPlateau(factor=0.1, patience=2, monitor='val_loss')
]

# Compilation et entraînement des modèles
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(images_normalized, labels_encoded, epochs=10, validation_split=0.2, callbacks=callbacks)

Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 147s 209ms/step - accuracy: 0.7109 - loss: 0.6009 - val_accuracy: 0.9652 - val_loss: 0.2866 - learning_rate: 0.0010
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 145s 210ms/step - accuracy: 0.9298 - loss: 0.2269 - val_accuracy: 0.7658 - val_loss: 0.4715 - learning_rate: 0.0010
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 148s 214ms/step - accuracy: 0.9330 - loss: 0.1991 - val_accuracy: 0.9048 - val_loss: 0.2565 - learning_rate: 0.0010
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 145s 210ms/step - accuracy: 0.9512 - loss: 0.1492 - val_accuracy: 0.9178 - val_loss: 0.2585 - learning_rate: 0.0010
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 131s 190ms/step - accuracy: 0.9584 - loss: 0.1276 - val_accuracy: 0.9006 - val_loss: 0.2985 - learning_rate: 0.0010
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 129s 187ms/step - accuracy: 0.9693 - loss: 0.0970 - val_accuracy: 0.9260 - val_loss: 0.2234 - learning_rate: 1.0000e-04
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 132s 191ms/step - ac

In [81]:
# Sauvegarde des poids des modèles
model.save_weights('model_weights.weights.h5')